In [144]:
globals().clear
import time
import pandas as pd
import numpy as np
import seaborn as sns
sns.set()
from datetime import datetime

In [145]:
# Install Libraries

# mount Google Drive
from google.colab import drive
drive.mount('/content/colabIntern')

%cd "/content/colabIntern/MyDrive/Capstone/Capstone Project/Collab"
!ls | wc -l

Drive already mounted at /content/colabIntern; to attempt to forcibly remount, call drive.mount("/content/colabIntern", force_remount=True).
/content/colabIntern/.shortcut-targets-by-id/1_cBH5e_TBWyCYfxXIK1yDOdiYjVxi7ae/Capstone Project/Collab
6


In [146]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler

import tensorflow       as tf
import keras.optimizers as op
from keras.callbacks import EarlyStopping

from tensorflow                     import keras
from tensorflow.keras.models        import Sequential
from tensorflow.keras.layers        import Dense
from tensorflow.keras.layers        import Dropout
from tensorflow.keras.optimizers    import Adam

In [147]:
!pip install tensorflow

In [148]:
# Load Dataset
df = pd.read_csv('df.csv')

In [149]:
# list of tickers for stocks in our data set. Sort the ticker list since our df will be alphabetically arranged.
tickers = ["AAPL", 'XOM', 'IBM', 'KO', 'CVX', 'BA', 'PFE', 'MSFT', 'T', 'WMT',
       'F', 'NFLX', 'JPM', 'MCD', 'GE', 'NVDA', 'JNJ', 'BAC', 'C', 'AMZN',
       'INTC', 'CSCO', 'TSLA', 'GOOGL', 'AMD', 'BABA', 'VZ', 'DIS',
       'META']
tickers.sort()

In [150]:
# For this specific trial drop PLTR since data is not complete
df = df[df.stock_ID != "PLTR"]

In [151]:
# without demographics
df = df.iloc[::,:74]

In [152]:
df['DATETIME']= pd.to_datetime(df['DATETIME'], format='%m/%d/%Y %H:%M')

In [153]:
# Sort by time so that the first 29 rows occupy the first time value for all the stocks.
# Fill in 0's for missing values for now.
df = df.set_index('DATETIME')
df = df.fillna(0)

In [154]:
# Select the length of the df ; For this file we need 7 months
# We use the first 3 months to make the first prediction, then shift window
# Then we will repeat this for the next 4 months (Hence total is 7 months needed)
df = df.sort_index().loc['2022-06-01':'2022-12-30']

In [155]:
#Set the DATETIME for fecha
DATETIME = df.index.values

In [156]:
# Sort dataframe such that it is both in sequential order, and also in alphabetical order for each day
#(i.e first entry for each time entry should be AAPL, and last should be XOM).
df.sort_values(["DATETIME", "stock_ID"], inplace=True)

In [157]:
num_companies = 29  # 29 companies in our dataset.
days = 5
step_rows  = 24 * num_companies * days # 24 time periods per day per stock
total_rows = len(df['2022-06-01':'2022-12-30'])  # Define total length to predict on
train_rows = len(df['2022-06-01':'2022-08-31'])  # Define length of training window


scale_X = MinMaxScaler()
model = Sequential()
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))  # initiating with 100 neurons
model.add(Dense(100, activation = 'relu', input_dim = df.iloc[:,1:-1].shape[1]))
model.add(Dropout(0.25))                                                          # adding dropout to avoid overfitting
model.add(Dense(1))                                                               # output layer

opt = Adam(amsgrad = True, lr = 0.001, beta_1 = 0.79, beta_2 = 0.999)             # using Adam optimizer, at a learning rate of 0.001
model.compile(loss = 'mse', optimizer = opt)                                      # compiling model
#es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=4)

result = pd.DataFrame(columns=['DATETIME', 'ACTUAL', 'PREDICTED', 'DIFFERENCE', 'TRAIN_DURATION'])

for i in range(0, total_rows - train_rows, step_rows):
    st = time.time()
    # 1. Obtain X and y
    train  = df.iloc[i:train_rows + i, 1:]
    test   = df.iloc[train_rows + i:train_rows + i + step_rows, 1:]
    X_train, y_train = train.iloc[:, 1:], train.iloc[:, 0]
    X_test, y_test = test.iloc[:, 1:], test.iloc[:, 0]



    # 2. Scale X
    X_train = scale_X.fit_transform(X_train)
    X_test = scale_X.transform(X_test)

    # 3. Fit and Predict
    model.fit(X_train, y_train)#, epochs=20, batch_size=256)#, callbacks=[es])
    y_hat = model.predict(X_test, verbose=False)
    et = time.time()

    # 4. Save data with prediction
    fecha = DATETIME[train_rows + i:train_rows + i + step_rows]
    datos = {
        'DATETIME': fecha.ravel(),
        'ACTUAL': y_test.ravel(),
        'PREDICTED': y_hat.ravel(),
        'DIFFERENCE': abs(y_hat.ravel() - y_test.ravel()),
        'TRAIN_DURATION': np.full(y_hat.ravel().shape[0], et - st)
    }
    data = pd.DataFrame(data=datos)
    result = pd.concat([result, data], ignore_index=True)

    print("Count Down:", int((total_rows - train_rows - i) / step_rows))
    # print(data.head())

1392/1392 [==============================] - 8s 5ms/step - loss: 5451.0264
Count Down: 16
1392/1392 [==============================] - 4s 3ms/step - loss: 4732.6265
Count Down: 15
1392/1392 [==============================] - 4s 3ms/step - loss: 4742.0312
Count Down: 14
1392/1392 [==============================] - 3s 2ms/step - loss: 4836.7119
Count Down: 13
1392/1392 [==============================] - 3s 2ms/step - loss: 4862.5063
Count Down: 12
1392/1392 [==============================] - 4s 3ms/step - loss: 4863.8301
Count Down: 11
1392/1392 [==============================] - 4s 3ms/step - loss: 4852.5781
Count Down: 10
1392/1392 [==============================] - 3s 2ms/step - loss: 4861.4302
Count Down: 9
1392/1392 [==============================] - 3s 2ms/step - loss: 4870.7778
Count Down: 8
1392/1392 [==============================] - 3s 2ms/step - loss: 4811.2173
Count Down: 7
1392/1392 [==============================] - 3s 2ms/step - loss: 4764.9033
Count Down: 6
1392/1392 [===

In [158]:
metrics_df = pd.DataFrame(columns = ['stock_ID', "RMSE", "MAPE", "MPE", "MTT"])

In [159]:
result[25::29]

,DATETIME,ACTUAL,PREDICTED,DIFFERENCE,TRAIN_DURATION
25,2022-09-01 09:30:00,274.66,107.703865,166.956135,12.511001
54,2022-09-01 09:45:00,273.62,102.915962,170.704038,12.511001
83,2022-09-01 10:00:00,271.36,108.000534,163.359466,12.511001
112,2022-09-01 10:15:00,273.14,106.388977,166.751023,12.511001
141,2022-09-01 10:30:00,269.54,109.281609,160.258391,12.511001
...,...,...,...,...,...
56256,2022-12-30 14:15:00,121.46,103.236931,18.223069,3.546824
56285,2022-12-30 14:30:00,121.30,101.220261,20.079739,3.546824
56314,2022-12-30 14:45:00,121.51,104.742851,16.767149,3.546824
56343,2022-12-30 15:00:00,122.42,107.165634,15.254366,3.546824


In [160]:
def mean_positive_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    error = np.mean(np.maximum((y_pred - y_true),0))
    return error

In [161]:
predicted, actual,duration=result['PREDICTED'],result['ACTUAL'],result['TRAIN_DURATION']

In [162]:
rmse = np.sqrt(mean_squared_error(result['ACTUAL'], result['PREDICTED']))
# Calculate MAPE %
mape = mean_absolute_percentage_error(result['ACTUAL'], result['PREDICTED']) * 100
# Calculate MPE %
mpe = mean_positive_error(result['ACTUAL'], result['PREDICTED'])
# Calculate MTT in seconds
mtt =duration.sum()

In [163]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mtt], index=['	stock_ID','MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [164]:
rmse,mape,mpe,metrics_df['MTT'].sum()

(68.52301971678558, 90.7697029099712, 26.86889110889708, 151.52742870648697)

In [131]:
new_rows = []
for i in range(len(tickers)):
    stock_result = result[i::29]
    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error(stock_result['ACTUAL'], stock_result['PREDICTED']))
    # Calculate MAPE %
    mape = mean_absolute_percentage_error(stock_result['ACTUAL'], stock_result['PREDICTED']) * 100
    # Calculate MPE %
    mpe = mean_positive_error(stock_result['ACTUAL'],stock_result['PREDICTED'])
    # Calculate MTT in seconds
    mtt = np.mean(stock_result['TRAIN_DURATION'])
    new_row = pd.Series([tickers[i], mape, mpe, mtt], index=['stock_ID',	'MAPE',	'MPE',	'MTT'])
    new_rows.append(new_row)
metrics_df = pd.DataFrame(new_rows)

In [132]:
#metrics_df

In [133]:
pd.DataFrame(metrics_df.drop(['stock_ID','MTT'],axis=1).mean()).T

,MAPE,MPE
0,21.105048,8.785318
